In [1]:
import torch

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\torchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',
    """
    When forty winters shall besiege thy brow,
    And dig deep trenches in thy beauty's field,
    Thy youth's proud livery so gazed on now,
    Will be a totter'd weed of small worth held:
    Then being asked, where all thy beauty lies,
    Where all the treasure of thy lusty days;
    To say, within thine own deep sunken eyes,
    Were an all-eating shame, and thriftless praise.
    How much more praise deserv'd thy beauty's use,
    If thou couldst answer 'This fair child of mine
    Shall sum my count, and make my old excuse,'
    Proving his beauty by succession thine!
    This were to be new made when thou art old,
    And see thy blood warm when thou feel'st it cold.
    """,
]

In [3]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [4]:
sentences = tokenize_corpus(corpus=corpus)

In [5]:
print(sentences)

[['he', 'is', 'a', 'king'], ['she', 'is', 'a', 'queen'], ['he', 'is', 'a', 'man'], ['she', 'is', 'a', 'woman'], ['warsaw', 'is', 'poland', 'capital'], ['berlin', 'is', 'germany', 'capital'], ['paris', 'is', 'france', 'capital'], ['When', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow,', 'And', 'dig', 'deep', 'trenches', 'in', 'thy', "beauty's", 'field,', 'Thy', "youth's", 'proud', 'livery', 'so', 'gazed', 'on', 'now,', 'Will', 'be', 'a', "totter'd", 'weed', 'of', 'small', 'worth', 'held:', 'Then', 'being', 'asked,', 'where', 'all', 'thy', 'beauty', 'lies,', 'Where', 'all', 'the', 'treasure', 'of', 'thy', 'lusty', 'days;', 'To', 'say,', 'within', 'thine', 'own', 'deep', 'sunken', 'eyes,', 'Were', 'an', 'all-eating', 'shame,', 'and', 'thriftless', 'praise.', 'How', 'much', 'more', 'praise', "deserv'd", 'thy', "beauty's", 'use,', 'If', 'thou', 'couldst', 'answer', "'This", 'fair', 'child', 'of', 'mine', 'Shall', 'sum', 'my', 'count,', 'and', 'make', 'my', 'old', "excuse,'", 'Proving

In [6]:
voc = []

In [7]:
for sentence in sentences:
     for word in sentence:
        if word not in voc:
            voc.append(word)

In [8]:
print(voc)

['he', 'is', 'a', 'king', 'she', 'queen', 'man', 'woman', 'warsaw', 'poland', 'capital', 'berlin', 'germany', 'paris', 'france', 'When', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow,', 'And', 'dig', 'deep', 'trenches', 'in', "beauty's", 'field,', 'Thy', "youth's", 'proud', 'livery', 'so', 'gazed', 'on', 'now,', 'Will', 'be', "totter'd", 'weed', 'of', 'small', 'worth', 'held:', 'Then', 'being', 'asked,', 'where', 'all', 'beauty', 'lies,', 'Where', 'the', 'treasure', 'lusty', 'days;', 'To', 'say,', 'within', 'thine', 'own', 'sunken', 'eyes,', 'Were', 'an', 'all-eating', 'shame,', 'and', 'thriftless', 'praise.', 'How', 'much', 'more', 'praise', "deserv'd", 'use,', 'If', 'thou', 'couldst', 'answer', "'This", 'fair', 'child', 'mine', 'Shall', 'sum', 'my', 'count,', 'make', 'old', "excuse,'", 'Proving', 'his', 'by', 'succession', 'thine!', 'This', 'were', 'to', 'new', 'made', 'when', 'art', 'old,', 'see', 'blood', 'warm', "feel'st", 'it', 'cold.']


In [9]:
word2idx = {w:idx for (idx, w) in enumerate(voc)}
inx2word = {idx:w for (idx, w) in enumerate(voc)}

In [10]:
# n-gram
CONTEXT_SIZE = 2
idx_pair = []
for sentence in sentences:
    for i in range(CONTEXT_SIZE, len(sentence)):
        context = [sentence[i - j - 1] for j in range(CONTEXT_SIZE)]
        target = sentence[i]
        idx_pair.append((context, target))

In [11]:
idx_pair[0]

(['is', 'he'], 'a')

In [12]:
class embedding(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim) -> None:
        super(embedding, self).__init__()
        self.tableW = torch.nn.Parameter(torch.randn((vocab_size, embedding_dim), requires_grad=True))
        self.embedding_dim = embedding_dim

    def forward(self, idx):
        return self.tableW[idx]



class NGramLanguageModeler(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size) -> None:
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = embedding(vocab_size, embedding_dim)
        self.linear1 = torch.nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = torch.nn.Linear(128, vocab_size)

    def forward(self, x):
        x = self.embeddings(x).view((1, -1)) # 這邊的想法是concat兩個tensor的向量後統合一個結果
        x = torch.nn.functional.relu(self.linear1(x))
        x = self.linear2(x)
        x = torch.nn.functional.log_softmax(x, dim=1)
        return x

In [13]:
losses = []
loss_function = torch.nn.NLLLoss()
model = NGramLanguageModeler(len(voc), 10, CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [17]:
model.embeddings(torch.tensor(data=[1,2,3]))

tensor([[-2.4958,  1.1488, -0.7197, -1.9668,  0.3909,  0.0794, -0.9873, -0.6977,
          0.1854, -0.1827],
        [ 0.2638, -2.2044,  0.7035, -0.4912,  1.7612,  0.3545,  2.4088,  1.5987,
         -1.5484,  0.7278],
        [ 1.1533,  2.8507,  1.6373,  0.4407, -0.8464, -0.3718, -0.6306,  0.1699,
         -0.7330,  0.0181]], grad_fn=<IndexBackward0>)

In [14]:
for epoch in range(100):
    total_loss = 0
    for context, target in idx_pair:
        context_idxs = torch.tensor([word2idx[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word2idx[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss) 

In [15]:
losses # The loss decreased every iteration over the training data!

[598.7933633327484,
 595.6715388298035,
 592.5725803375244,
 589.4955089092255,
 586.4399955272675,
 583.4038963317871,
 580.3849759101868,
 577.3839056491852,
 574.3989176750183,
 571.4298176765442,
 568.4764256477356,
 565.5374565124512,
 562.6128051280975,
 559.701164484024,
 556.8011193275452,
 553.9107966423035,
 551.0319902896881,
 548.1663072109222,
 545.3142218589783,
 542.476115822792,
 539.6519093513489,
 536.8428168296814,
 534.0483647584915,
 531.2678909301758,
 528.5019017457962,
 525.7524298429489,
 523.0173176527023,
 520.2961111068726,
 517.590426325798,
 514.897775888443,
 512.2192373275757,
 509.55303502082825,
 506.89947444200516,
 504.25756990909576,
 501.62536519765854,
 499.00207644701004,
 496.38615822792053,
 493.7772195339203,
 491.1718570590019,
 488.5691973567009,
 485.9689711332321,
 483.3662385940552,
 480.76508498191833,
 478.1642761826515,
 475.5609430074692,
 472.95479705929756,
 470.34450829029083,
 467.72822764515877,
 465.1058475971222,
 462.479246199